In [ ]:
import keras
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import tensorflow as tf

In [ ]:
devices = tf.config.experimental.list_physical_devices('GPU')
if devices:
    # If GPUs are available, enable memory growth on them
    for device in devices:
        tf.config.experimental.set_memory_growth(device, True)
    # Set the first GPU as visible
    tf.config.experimental.set_visible_devices(devices[0], 'GPU')
else:
    # If no GPUs are available, list all CPUs
    devices = tf.config.experimental.list_physical_devices('CPU')
    # Set the first CPU as visible (typically there's only one)
    tf.config.experimental.set_visible_devices(devices[0], 'CPU')
    
# plt increases the default figure size and
plt.rcParams['figure.figsize'] = [8, 8]
plt.rcParams['figure.dpi'] = 125
# tight_layout no axis 
plt.rcParams['axes.axisbelow'] = True

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)


In [ ]:
class DCGAN:
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = keras.optimizers.Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = keras.layers.Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = keras.Model(z, valid)
        self.combined.compile(loss="binary_crossentropy", optimizer=optimizer)

    def build_generator(self):
        model = keras.models.Sequential(
            [
                keras.layers.InputLayer(input_shape=(self.latent_dim,)),
                keras.layers.Dense(128 * 7 * 7, activation="relu"),
                keras.layers.Reshape((7, 7, 128)),
                keras.layers.UpSampling2D(),
                keras.layers.Conv2D(128, kernel_size=3, padding="same"),
                keras.layers.BatchNormalization(momentum=0.8),
                keras.layers.Activation("relu"),
                keras.layers.UpSampling2D(),
                keras.layers.Conv2D(64, kernel_size=3, padding="same"),
                keras.layers.BatchNormalization(momentum=0.8),
                keras.layers.Activation("relu"),
                keras.layers.Conv2D(self.channels, kernel_size=3, padding="same"),
                keras.layers.Activation("tanh"),
            ]
        )
        noise = keras.layers.Input(shape=(self.latent_dim,))
        img = model(noise)

        return keras.Model(noise, img)

    def build_discriminator(self):
        model = keras.models.Sequential(
            [
                keras.layers.InputLayer(shape=self.img_shape),
                keras.layers.Conv2D(32, kernel_size=3, strides=2, padding="same"),
                keras.layers.LeakyReLU(alpha=0.2),
                keras.layers.Dropout(0.25),
                keras.layers.Conv2D(64, kernel_size=3, strides=2, padding="same"),
                keras.layers.ZeroPadding2D(padding=((0, 1), (0, 1))),
                keras.layers.BatchNormalization(momentum=0.8),
                keras.layers.LeakyReLU(alpha=0.2),
                keras.layers.Dropout(0.25),
                keras.layers.Conv2D(128, kernel_size=3, strides=2, padding="same"),
                keras.layers.BatchNormalization(momentum=0.8),
                keras.layers.LeakyReLU(alpha=0.2),
                keras.layers.Dropout(0.25),
                keras.layers.Conv2D(256, kernel_size=3, strides=1, padding="same"),
                keras.layers.BatchNormalization(momentum=0.8),
                keras.layers.LeakyReLU(alpha=0.2),
                keras.layers.Dropout(0.25),
                keras.layers.Flatten(),
                keras.layers.Dense(1, activation="sigmoid"),
            ]
        )
        img = keras.layers.Input(shape=self.img_shape)
        validity = model(img)

        return keras.Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):
        # Load the dataset
        (X_train, _), (_, _) = keras.datasets.mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.0
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):
            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise, verbose=0)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print(
                f"{epoch} [D loss: {d_loss[0]:.2f}, acc.:{d_loss[1]:.2%}] [G loss: {np.mean(g_loss):.2f}]"
            )

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise, verbose=0)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        Path("images/DCGAN").mkdir(parents=True, exist_ok=True)

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap="gray")
                axs[i, j].axis("off")
                cnt += 1
        fig.savefig(f"images/DCGAN/mnist_{epoch}.png")
        plt.close()

In [ ]:
if __name__ == "__main__":
    dcgan = DCGAN()
    dcgan.train(epochs=400, batch_size=32, save_interval=50)